1. x Test daten abspeichern und hier einlesen
2. Ereigniskenndaten einlesen von csv
3. zu den listenobjekten jeweils eventdaten hinzufügen (Dauer, hN, imax_60, Jährlichkeit)
4. Benchmark eventsauswählen
5. load in_scaler und out_scaler
6. testdaten sequenzieren und normalisieren
7. daten vorhersagen
8. predictions mit delta time index versehen
9. Auswahl plotten mit delta time
    - wie plotten? evtl niederschlag mit balken von oben und predictions + actual ganglinie von unten
    - ereigniss kenndaten angeben (Dauer, hN, imax_60, Jährlichkeit)

Auswertung aller Sequencen
- MAE
- RMSE
- Scatterplott Actual vs Fitted
- Boxplot mit quantilen für jeden Vorhersagezeitschritt
- siehe Plots burrichter




In [1]:
# Load Relevant Libraries, the model and the test data
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import pandas as pd
from tensorflow.keras.models import model_from_json
import joblib
import pickle
import os
from modules.sequence_and_normalize import sequence_data, sequence_sample_random, sequence_list
from sklearn.preprocessing import MinMaxScaler


# Assign all relevant paths
model_folder = '05_models\\Gievenbeck_SingleNode_LSTM_20240328'
model_name = "Gievenbeck_SingleNode_LSTM_20240328"
model_path = os.path.join(model_folder, f'{model_name}.json')
weights_path = os.path.join(model_folder, f'{model_name}.weights.h5')
in_scaler_path = os.path.join(model_folder, 'in_scaler.pkl')
out_scaler_path = os.path.join(model_folder, 'out_scaler.pkl')
test_data_path = os.path.join(model_folder, 'test_data')
# Saving model design to JSON

# load json and create model
json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(weights_path)

# Load the scalers
in_scaler = joblib.load(in_scaler_path)
out_scaler = joblib.load(out_scaler_path)

# Load the test data
with open(test_data_path, 'rb') as file:
    test_data = pickle.load(file)

print("Loaded model from disk")

Loaded model from disk


In [2]:
# Load events data
events_df = pd.read_csv('02_input_data/events_FMO.csv')

# sequence data to list structure
lag = int(3 * 60 / 5)
delay = 0
p_steps = 6

in_vars=['duration', 'p']
out_vars=['Q_out']

seq_test = sequence_list(test_data, in_vars=in_vars, out_vars=out_vars, in_scaler=in_scaler, 
                                  out_scaler=out_scaler, lag=lag, delay=delay, prediction_steps=p_steps)


In [3]:
print(seq_test[0])
# print(events_df)

# print(12.65*60/5)


[{'name': 'Gievenbeck_2016-02-22 03 40 00_hN12 65', 'duration': 765, 'total precipitation': 12.649992823600769, 'max intensity': 15.83999605178833, 'intervall': 5}, array([[[0.        , 0.        ],
        [0.00106952, 0.        ],
        [0.00213904, 0.        ],
        ...,
        [0.03529412, 0.00124402],
        [0.03636364, 0.00698564],
        [0.03743316, 0.01043061]],

       [[0.00106952, 0.        ],
        [0.00213904, 0.        ],
        [0.00320856, 0.        ],
        ...,
        [0.03636364, 0.00698564],
        [0.03743316, 0.01043061],
        [0.03850267, 0.0105263 ]],

       [[0.00213904, 0.        ],
        [0.00320856, 0.        ],
        [0.00427807, 0.        ],
        ...,
        [0.03743316, 0.01043061],
        [0.03850267, 0.0105263 ],
        [0.03957219, 0.00430622]],

       ...,

       [[0.14224599, 0.0060287 ],
        [0.14331551, 0.00727272],
        [0.14438503, 0.0075598 ],
        ...,
        [0.17754011, 0.        ],
        [0.17860